In [1]:
from pyspark.sql import SparkSession , types, functions,udf
from pyspark.sql.functions import Column,udf
# description: id,season,city,date,team1,team2,toss_winner,
# toss_decision,result,dl_applied,winner,win_by_runs,win_by_wickets,player_of_match,venue,umpire1,umpire2,umpire3

# Creating Spark Session

In [2]:

spark=SparkSession.builder.master("local").appName("IPL").getOrCreate()

# Reading the CSV file


In [4]:
df=spark.read.csv("IPL-Match-Details.csv")
df.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: string (nullable = true)
 |-- _c3: string (nullable = true)
 |-- _c4: string (nullable = true)
 |-- _c5: string (nullable = true)
 |-- _c6: string (nullable = true)
 |-- _c7: string (nullable = true)
 |-- _c8: string (nullable = true)
 |-- _c9: string (nullable = true)
 |-- _c10: string (nullable = true)
 |-- _c11: string (nullable = true)
 |-- _c12: string (nullable = true)
 |-- _c13: string (nullable = true)
 |-- _c14: string (nullable = true)
 |-- _c15: string (nullable = true)
 |-- _c16: string (nullable = true)
 |-- _c17: string (nullable = true)



# Giving Column names to the Data

In [7]:
total_df=df.select(df[0].alias("ID"),df[1].alias("Season"),df[2].alias("City"),df[3].alias("Date"),df[4].alias("Team1"),df[5].alias("Team2"),df[6].alias("toss_winner"),df[7].alias("toss_decision"),df[8].alias("result"),df[9].alias("dl_applied"),df[10].alias("winner"),df[11].alias("win_by_runs"),df[12].alias("win_by_wickets"),df[13].alias("player_of_match"),df[14].alias("venue"),df[15].alias("umpire1"),df[16].alias("umpire2"),df[17].alias("umpire3"))

In [8]:
total_df.printSchema()

root
 |-- ID: string (nullable = true)
 |-- Season: string (nullable = true)
 |-- City: string (nullable = true)
 |-- Date: string (nullable = true)
 |-- Team1: string (nullable = true)
 |-- Team2: string (nullable = true)
 |-- toss_winner: string (nullable = true)
 |-- toss_decision: string (nullable = true)
 |-- result: string (nullable = true)
 |-- dl_applied: string (nullable = true)
 |-- winner: string (nullable = true)
 |-- win_by_runs: string (nullable = true)
 |-- win_by_wickets: string (nullable = true)
 |-- player_of_match: string (nullable = true)
 |-- venue: string (nullable = true)
 |-- umpire1: string (nullable = true)
 |-- umpire2: string (nullable = true)
 |-- umpire3: string (nullable = true)



# Requirement 1> Which stadium is best suitable for first batting (%)

In [10]:

stadium_analysis_df=total_df.select("toss_winner", "win_by_runs","win_by_wickets","venue")
print(stadium_analysis_df.show())
# print(type(stadium_analysis_df))

+--------------------+-----------+--------------+--------------------+
|         toss_winner|win_by_runs|win_by_wickets|               venue|
+--------------------+-----------+--------------+--------------------+
|Royal Challengers...|        140|             0|M Chinnaswamy Sta...|
| Chennai Super Kings|         33|             0|Punjab Cricket As...|
|    Rajasthan Royals|          0|             9|    Feroz Shah Kotla|
|      Mumbai Indians|          0|             5|    Wankhede Stadium|
|     Deccan Chargers|          0|             5|        Eden Gardens|
|     Kings XI Punjab|          0|             6|Sawai Mansingh St...|
|     Deccan Chargers|          0|             9|Rajiv Gandhi Inte...|
|      Mumbai Indians|          6|             0|MA Chidambaram St...|
|    Rajasthan Royals|          0|             3|Rajiv Gandhi Inte...|
|      Mumbai Indians|         66|             0|Punjab Cricket As...|
|    Rajasthan Royals|          0|             7|M Chinnaswamy Sta...|
|Kolka

In [11]:
#find number of first bat win  for each unique venue
venue_firstBat=stadium_analysis_df.rdd.map(lambda x:(x[3],int(x[1]))).filter(lambda x:x[1] >0)


In [16]:
#List of all the stadiums where the first BAT team won
venue_firstBat.collect()

[('M Chinnaswamy Stadium', 140),
 ('Punjab Cricket Association Stadium, Mohali', 33),
 ('MA Chidambaram Stadium, Chepauk', 6),
 ('Punjab Cricket Association Stadium, Mohali', 66),
 ('M Chinnaswamy Stadium', 13),
 ('Feroz Shah Kotla', 10),
 ('Sawai Mansingh Stadium', 45),
 ('Punjab Cricket Association Stadium, Mohali', 9),
 ('Dr DY Patil Sports Academy', 29),
 ('Eden Gardens', 5),
 ('MA Chidambaram Stadium, Chepauk', 18),
 ('Rajiv Gandhi International Stadium, Uppal', 23),
 ('Eden Gardens', 23),
 ('Punjab Cricket Association Stadium, Mohali', 41),
 ('Feroz Shah Kotla', 12),
 ('Feroz Shah Kotla', 6),
 ('Sawai Mansingh Stadium', 65),
 ('Rajiv Gandhi International Stadium, Uppal', 25),
 ('Eden Gardens', 3),
 ('Wankhede Stadium', 1),
 ('MA Chidambaram Stadium, Chepauk', 14),
 ('MA Chidambaram Stadium, Chepauk', 10),
 ('M Chinnaswamy Stadium', 3),
 ('Wankhede Stadium', 105),
 ('Newlands', 19),
 ('Newlands', 75),
 ("St George's Park", 92),
 ('Kingsmead', 11),
 ('Newlands', 24),
 ('Kingsmead',

In [34]:
# Creating a tuple with Stadium name and int[1], this int[1] we will add to find the total count per stadium
venue_firstBat1=venue_firstBat.map(lambda x:(x[0],1)).reduceByKey(lambda a,b:a+b).sortBy(lambda x:x[1],ascending=False)
result1=venue_firstBat1.collect()
print(*result1, sep = "\n")  # just to print the result line by line for better visibility


('MA Chidambaram Stadium, Chepauk', 30)
('Wankhede Stadium', 25)
('M Chinnaswamy Stadium', 24)
('Feroz Shah Kotla', 24)
('Eden Gardens', 22)
('Punjab Cricket Association Stadium, Mohali', 15)
('Rajiv Gandhi International Stadium, Uppal', 14)
('Subrata Roy Sahara Stadium', 11)
('Sawai Mansingh Stadium', 10)
('Kingsmead', 9)
('Dr DY Patil Sports Academy', 7)
('Dr. Y.S. Rajasekhara Reddy ACA-VDCA Cricket Stadium', 7)
('Brabourne Stadium', 6)
('Sardar Patel Stadium, Motera', 6)
('Himachal Pradesh Cricket Association Stadium', 5)
('Newlands', 4)
('SuperSport Park', 4)
('Barabati Stadium', 4)
("St George's Park", 3)
('New Wanderers Stadium', 3)
('Nehru Stadium', 3)
('Sheikh Zayed Stadium', 3)
('Dubai International Cricket Stadium', 3)
('Maharashtra Cricket Association Stadium', 3)
('Punjab Cricket Association IS Bindra Stadium, Mohali', 3)
('Buffalo Park', 2)
('Vidarbha Cricket Association Stadium, Jamtha', 2)
('Shaheed Veer Narayan Singh International Stadium', 2)
('JSCA International Stadi

# Requirement 2> Which stadium is best suitable for first bowling (%)

In [36]:
#find number of first bowl win  for each unique venue
venue_firstBowl=stadium_analysis_df.rdd.map(lambda x:(x[3],int(x[2]))).filter(lambda x:x[1] >0)

In [38]:
#List of all the stadiums where the first BAT team won
venue_firstBowl.collect()

[('Feroz Shah Kotla', 9),
 ('Wankhede Stadium', 5),
 ('Eden Gardens', 5),
 ('Sawai Mansingh Stadium', 6),
 ('Rajiv Gandhi International Stadium, Uppal', 9),
 ('Rajiv Gandhi International Stadium, Uppal', 3),
 ('M Chinnaswamy Stadium', 7),
 ('MA Chidambaram Stadium, Chepauk', 9),
 ('Dr DY Patil Sports Academy', 10),
 ('Punjab Cricket Association Stadium, Mohali', 4),
 ('Eden Gardens', 7),
 ('Rajiv Gandhi International Stadium, Uppal', 7),
 ('MA Chidambaram Stadium, Chepauk', 8),
 ('Rajiv Gandhi International Stadium, Uppal', 5),
 ('Sawai Mansingh Stadium', 8),
 ('M Chinnaswamy Stadium', 6),
 ('MA Chidambaram Stadium, Chepauk', 7),
 ('Dr DY Patil Sports Academy', 7),
 ('Feroz Shah Kotla', 4),
 ('Sawai Mansingh Stadium', 8),
 ('M Chinnaswamy Stadium', 9),
 ('Sawai Mansingh Stadium', 3),
 ('Punjab Cricket Association Stadium, Mohali', 9),
 ('Wankhede Stadium', 9),
 ('Wankhede Stadium', 8),
 ('M Chinnaswamy Stadium', 5),
 ('Eden Gardens', 6),
 ('Punjab Cricket Association Stadium, Mohali', 

In [41]:
# Creating a tuple with Stadium name and int[1], this int[1] we will add to find the total count per stadium
venue_firstBowl1=venue_firstBowl.map(lambda x:(x[0],1)).reduceByKey(lambda a,b:a+b).sortBy(lambda x:x[1],ascending=False)
result2=venue_firstBowl1.collect()
print(*result2, sep = "\n")  # just to print the result line by line for better visibility


('Eden Gardens', 32)
('M Chinnaswamy Stadium', 31)
('Feroz Shah Kotla', 28)
('Rajiv Gandhi International Stadium, Uppal', 26)
('Wankhede Stadium', 24)
('Sawai Mansingh Stadium', 23)
('Punjab Cricket Association Stadium, Mohali', 20)
('MA Chidambaram Stadium, Chepauk', 17)
('Dr DY Patil Sports Academy', 10)
('SuperSport Park', 8)
('Kingsmead', 6)
('Subrata Roy Sahara Stadium', 6)
('New Wanderers Stadium', 5)
('Sardar Patel Stadium, Motera', 5)
('Brabourne Stadium', 5)
('JSCA International Stadium Complex', 5)
('Maharashtra Cricket Association Stadium', 5)
("St George's Park", 4)
('Himachal Pradesh Cricket Association Stadium', 4)
('Dr. Y.S. Rajasekhara Reddy ACA-VDCA Cricket Stadium', 4)
('Shaheed Veer Narayan Singh International Stadium', 4)
('Sharjah Cricket Stadium', 4)
('Dubai International Cricket Stadium', 4)
('Punjab Cricket Association IS Bindra Stadium, Mohali', 4)
('Saurashtra Cricket Association Stadium', 4)
('Barabati Stadium', 3)
('Sheikh Zayed Stadium', 3)
('Newlands', 2)


In [44]:

#join bat first and bowl first rdds
best_venue_analysis=venue_firstBat1.join(venue_firstBowl1)
# print(best_venue_analysis.collect())
# print(type(best_venue_analysis))

#calculate the percentage of first bat win by total matches with results
# no.of.bat.first.wins divide by total

best_venue_analysis1=best_venue_analysis.map(lambda x:(x[0],(x[1][0])/(x[1][0]+x[1][1])*100))

# "Below are the best venue to BAT first"


In [46]:

result3=best_venue_analysis1.collect()
print(*result3, sep = "\n")  # just to print the result line by line for better visibility

('MA Chidambaram Stadium, Chepauk', 63.829787234042556)
('Wankhede Stadium', 51.02040816326531)
('Punjab Cricket Association Stadium, Mohali', 42.857142857142854)
('Rajiv Gandhi International Stadium, Uppal', 35.0)
('Sawai Mansingh Stadium', 30.303030303030305)
('Dr. Y.S. Rajasekhara Reddy ACA-VDCA Cricket Stadium', 63.63636363636363)
('Brabourne Stadium', 54.54545454545454)
('Sardar Patel Stadium, Motera', 54.54545454545454)
('Newlands', 66.66666666666666)
('Barabati Stadium', 57.14285714285714)
('Nehru Stadium', 60.0)
('Sheikh Zayed Stadium', 50.0)
('Dubai International Cricket Stadium', 42.857142857142854)
('Shaheed Veer Narayan Singh International Stadium', 33.33333333333333)
('De Beers Diamond Oval', 33.33333333333333)
('OUTsurance Oval', 50.0)
('Saurashtra Cricket Association Stadium', 20.0)
('M Chinnaswamy Stadium', 43.63636363636363)
('Feroz Shah Kotla', 46.15384615384615)
('Eden Gardens', 40.74074074074074)
('Subrata Roy Sahara Stadium', 64.70588235294117)
('Kingsmead', 60.0)
